# Task 3.2: Embedding Hidden Messages 

# START of code personally written without assistance

This notebook conducts audio steganography using the least significant bit (LSB) method. It embeds hidden 
messages based on the LSB audio steganography method to embed the secret message ‘An 
eye for an eye makes the whole world blind’. It also includes an algorithm that performs the opposite operation

## 1 Import Necessary Packages

In [1]:
import wave

## 2 Define convert_Str_to_Bits Function

Each character in the input string is transformed into its binary form. The binary representation is padded to ensure it is 8 bits long. A '#' character is appended to the message to indicate the end. The resulting binary string is converted into a list of integers (map object), effectively representing the binary bits of the input message plus the delimiter.

In [2]:
def convert_Str_to_Bits(message: str) -> map:
    """
    Converts a string into a list of bits.

    The function transforms each character in the input string into its 
    binary representation and then returns these as a list of integers.

    :param message: The string message to convert into bits.
    :return: A map object containing integers representing the binary bits 
             of the input message plus a '#' character as a delimiter.
    """
    # Append '#' to the message to signify the end
    # Convert each character to its binary form and pad zeros to ensure 8 bits
    # Join these binary strings and convert them to a map of integers
    return map(int, "".join([bin(ord(char))[2:].rjust(8, '0') for char in message + '#']))

## 3 Define merge_Frames_and_LSBs Function
Encodes a hidden message into an audio file by manipulating the least significant bit (LSB) of each byte in the audio frames. The function iterates over each byte in frame_bytes alongside each bit in bits. It uses bitwise operations to replace the LSB of each audio byte with the corresponding bit from the message. This is achieved by using 0xFE (11111110 in binary) to mask out the current LSB from each audio byte and adding the LSB bit from the message to this masked byte using 0x1. It returns a new bytes object that represents the audio frames with the message bits embedded in their LSB positions.

In [3]:
def merge_Frames_and_LSBs(frame_bytes: bytes, bits: map) -> bytes:
    """
    Merges audio frame bytes with LSB bits from a message.

    This function combines the least significant bit (LSB) of each byte in 
    the audio frames with the corresponding bit from the message, effectively
    encoding the message into the audio file.

    :param frame_bytes: The bytes of the audio frame to be modified.
    :param bits: A map object containing the LSB bits to embed into the audio frames.
    :return: A new bytes object representing the modified audio frames with the message embedded.
    """
    # Create new bytes by replacing the LSB of each audio byte with the message bit
    # 0xFE masks out the LSB of the frame byte, 0x1 masks the bit value
    return bytes((0xFE & frame) + (0x1 & bit) for frame, bit in zip(frame_bytes, bits))

## 4 Define encode_File function
Encodes a hidden message within the frames of an audio file, ensuring that the alteration is subtle and imperceptible during playback. The function opens the original audio file and retrieves its frame bytes and audio parameters, then converts the input message into a sequence of binary bits using the convert_Str_to_Bits function. It utilizes merge_Frames_and_LSBs to replace the LSBs of the audio frame bytes with the message bits, effectively embedding the message and writes the modified frame bytes to a new audio file with the specified new_filename, preserving the original audio parameters.


In [4]:
def encode_File(filename: str, message: str, new_filename: str) -> None:
    """
    Encodes a message into an audio file using LSB steganography.

    This function takes an input audio file and a message, then produces 
    a new audio file with the message embedded in its frames via LSB 
    (Least Significant Bit) manipulation.

    :param filename: The name of the original audio file.
    :param message: The message to encode into the audio file.
    :param new_filename: The name for the new audio file with the hidden message.
    """
    # Open the input audio file to read its frames and parameters
    with wave.open(filename, mode='rb') as music:
        frame_bytes = bytes(music.readframes(music.getnframes()))
        music_params = music.getparams()

    # Convert the message into bits
    bits = convert_Str_to_Bits(message)
    # Merge the message bits into the audio frames
    new_frame_bytes = merge_Frames_and_LSBs(frame_bytes, bits)

    # Write the modified frames to a new audio file
    with wave.open(new_filename, 'wb') as music:
        music.setparams(music_params)
        music.writeframes(new_frame_bytes)

## 5 Define extract_LSB_from_File Function
The function opens the specified audio file in 'read binary' mode using the wave module. 
It reads all the frames of the audio file and converts them into bytes. 
Using a bitwise AND operation with 0x1, the function isolates the LSB from each byte of the audio frames.
It returns a list of integers, each representing the LSB of a frame byte. 
This list can be used to decode any hidden message embedded using the LSB method.


In [5]:
def extract_LSB_from_File(filename: str) -> list:
    """
    Extracts the least significant bits (LSBs) from an audio file.

    :param filename: The name of the audio file to extract LSBs from.
    :return: A list of integers representing the LSBs from the audio frames.
    """
    with wave.open(filename, mode='rb') as music:
        frame_bytes = bytes(music.readframes(music.getnframes()))
    # Use bitwise AND to extract the LSB from each byte
    return [frame_byte & 0x1 for frame_byte in frame_bytes]

## 6 Define merge_8Bits_to_Char Function
The function first creates a binary string by joining the list of bits (bytes8) into a single string of '0's and '1's.
The binary string is prefixed with '0b' to indicate that it is a binary number.
The eval function converts this binary string into its equivalent integer, which is then transformed into a character using the chr function.
The final character is returned, representing the input bits as a text character.
This method is crucial in decoding messages hidden with the LSB method in audio steganography.

In [6]:
def merge_8Bits_to_Char(bytes8: list) -> str:
    """
    Converts a list of 8 bits into a character.

    :param bytes8: A list of 8 bit integers.
    :return: The character represented by the 8 bits.
    """
    # Convert 8 bits into a binary string and then to a character
    return chr(eval('0b' + "".join(map(str, bytes8))))

## 7 Define convert_LSB_Bytes_to_Chars Function
The function starts by creating an empty list called chars to hold the decoded characters.
The function then processes the input list lsb_bytes in chunks or sets of 8 bits, which correspond to one character. This is done using a list comprehension that slices the LSBs.
For each 8-bit chunk, the function merge_8Bits_to_Char is used to convert these bits into a character.
The process continues until it encounters the character '#', which serves as a delimiter marking the end of the hidden message.
Finally, the function returns a list of characters that compose the hidden message extracted from the LSBs.


In [7]:
def convert_LSB_Bytes_to_Chars(lsb_bytes: list) -> list:
    """
    Converts a list of LSB bytes into a list of characters.

    :param lsb_bytes: A list of LSBs from which to extract characters.
    :return: A list of characters extracted from the LSBs.
    """
    chars = []
    # Group bits into sets of 8, convert to char, and append to the char list
    for bits8 in [lsb_bytes[i: i + 8] for i in range(0, len(lsb_bytes), 8)]:
        each_char = merge_8Bits_to_Char(bits8)
        # Stop when the delimiter '#' is encountered
        if each_char != '#':
            chars.append(each_char)
        else:
            break
    return chars

## 8 Define decode_File Function
The function calls the extract_LSB_from_File function to retrieve the LSBs from each byte of the specified audio file, filename. 
The extracted bits are then passed to the convert_LSB_Bytes_to_Chars function, which groups the bits into sets of 8, converting each set into a character to form a list of decoded characters.
The list of characters is joined together into a single string, representing the decoded hidden message. 
Finally, the function returns this decoded string, which is the hidden message initially embedded within the audio file using LSB steganography techniques.


In [8]:
def decode_File(filename: str) -> str:
    """
    Decodes a hidden message from an audio file.

    :param filename: The name of the audio file to decode the message from.
    :return: The decoded message as a string.
    """
    # Extract LSBs and convert them to characters
    bytes_extracted = extract_LSB_from_File(filename)
    chars_extracted = convert_LSB_Bytes_to_Chars(bytes_extracted)
    # Join the character list into a single string
    return "".join(chars_extracted)

## 9 Embed and Recover Hidden Messages
The SECRET_MESSAGE variable holds the message "Father Christmas does not exist" that you want to embed into an audio file. 
The encode_File function is invoked, embedding the SECRET_MESSAGE into the SOURCE_FILE. 
This creates a new audio file, MODIFIED_FILE, where the message is hidden using the least significant bit (LSB) method. 
The decode_File function extracts the hidden message from the MODIFIED_FILE. It uses the LSB extraction and conversion techniques to reconstruct the original message.
Finally, the code prints the extracted hidden message to the console, confirming the process's success.


In [9]:
# Secret message to be encoded in the audio file
SECRET_MESSAGE = 'Father Christmas does not exist'

# Source and modified file names
SOURCE_FILE = "Ex3_sound5.wav"
MODIFIED_FILE = "modified_" + SOURCE_FILE

# Encode the secret message into the source audio file
encode_File(SOURCE_FILE, SECRET_MESSAGE, MODIFIED_FILE)

# Decode the message from the modified audio file
decoded = decode_File(MODIFIED_FILE)

# Print the hidden message
print(f'Hidden message: {decoded}')

Hidden message: Father Christmas does not exist


# End of code personally written without assistance